In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
from fastai import *
from fastai.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, roc_auc_score
import torch
import torch.nn as nn
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = Config.data_path()
lang = 'vi'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
bs=64

In [3]:
train_df = pd.read_csv('stock_data/data_clean.csv', sep='#')
train_df.loc[pd.isna(train_df.text),'text']='NA'
train_df['text'] = train_df['text'].str.lower()
label_encoder = LabelEncoder() 
train_df['sentiment_final']= label_encoder.fit_transform(train_df['sentiment_final']) 
train_df

,text,sentiment_final
0,xin thông báo khi nào hết dịch corona thì lúc ...,1
1,5 mã trắng cửa bán rồi thì flc tí nữa thôi là ...,1
2,hihi theo a quyết đến hết mùa dịch chắc cũng p...,1
3,"mai tây (etfs) nó xả hàng ros, mà a qc còn chơ...",0
4,"art, hai, klf lái con chốt, a e vô đỡ thấy thư...",0
...,...,...
425,tin vĩ mô:.- bộ tc gia tăng gói hỗ trợ một số ...,1
426,t2 này khả năng thoát dớp nhưng về lâu dài phe...,0
427,giảm thêm 700 đáy chắc,0
428,"cuối tuần ngập tràn tin xấu.cá mập, đội lái đã...",0


In [4]:
data = pd.read_csv('stock_data/data.csv', sep=',')
data.loc[pd.isna(data.content),'content']='NA'
data['content'] = data['content'].str.lower()
data

,thread_url,thread_name,title?,content,created_date,sentiment,keywords
0,http://f319.com/threads/xin-chuc-mung-co-dong-...,"Xin chúc mừng cổ đông của a QC: GAB, ART, KLF,...",Y,"xin chúc mừng cổ đông của a qc: gab, art, klf,...",18/03/2020,P,chúc mừng
1,http://f319.com/threads/xin-chuc-mung-co-dong-...,"Xin chúc mừng cổ đông của a QC: GAB, ART, KLF,...",N,chúc mừng ace nào theo đuôi mình và đã có hàng...,18/03/2020,P,"chúc mừng, hưởng thành quả"
2,http://f319.com/threads/xin-chuc-mung-co-dong-...,"Xin chúc mừng cổ đông của a QC: GAB, ART, KLF,...",N,"có lãi lo chốt đi, ở đó múa bên băng 3 nốt nhạ...",18/03/2020,NaN,NaN
3,http://f319.com/threads/xin-chuc-mung-co-dong-...,"Xin chúc mừng cổ đông của a QC: GAB, ART, KLF,...",N,anh quýt kotex-index thật vi diệu. tím thủy ch...,18/03/2020,NaN,NaN
4,http://f319.com/threads/xin-chuc-mung-co-dong-...,"Xin chúc mừng cổ đông của a QC: GAB, ART, KLF,...",N,ros trần phát tây lông bán như phá mả ..em khô...,18/03/2020,NaN,NaN
...,...,...,...,...,...,...,...
27213,http://f319.com/threads/sang-nay-khong-ghi-nha...,Sáng nay không ghi nhận thêm ca nhiễm nCoV,Y,sáng nay không ghi nhận thêm ca nhiễm ncov,5/4/2020,NaN,NaN
27214,http://f319.com/threads/sang-nay-khong-ghi-nha...,Sáng nay không ghi nhận thêm ca nhiễm nCoV,N,"chủ nhật, 5/4/2020, 06:17 (gmt+7)..sáng nay kh...",5/4/2020,NaN,NaN
27215,http://f319.com/threads/sang-nay-khong-ghi-nha...,Sáng nay không ghi nhận thêm ca nhiễm nCoV,N,tuyệt vời việt nam,5/4/2020,NaN,NaN
27216,http://f319.com/threads/sang-nay-khong-ghi-nha...,Sáng nay không ghi nhận thêm ca nhiễm nCoV,N,múc,5/4/2020,NaN,NaN


In [5]:
data_lm = (TextList.from_df(data, path, cols='content')
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=4))

In [6]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0)

In [7]:
lr = 1e-3
lr *= bs/48

In [8]:
learn_lm.fit_one_cycle(2, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.655009,4.309606,0.267420,00:35
1,4.475599,4.196741,0.276859,00:36


In [9]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.347687,4.089919,0.285774,00:46
1,4.194607,3.953220,0.300888,00:46
2,4.067661,3.841664,0.314614,00:47
3,3.942392,3.761452,0.324189,00:46
4,3.845918,3.701095,0.331974,00:47
5,3.755334,3.656090,0.337956,00:47
6,3.761018,3.627942,0.342092,00:47
7,3.712162,3.608828,0.344456,00:46
8,3.662963,3.601228,0.345704,00:45
9,3.636343,3.601158,0.345846,00:46


In [10]:
learn_lm.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(11832, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(11832, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=11832, bias=True)
    (output_dp): RNNDropout()
  )
)

In [11]:
learn_lm.save('stock_sentiment_finetuned')
learn_lm.save_encoder('stock_sentiment_finetuned_enc')

In [12]:
data_clas = (TextList.from_df(train_df, path, vocab=data_lm.vocab, cols='text')
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='sentiment_final')
    .databunch(bs=bs, num_workers=1))
data_clas

TextClasDataBunch;

Train: LabelList (387 items)
x: TextList
xxbos xin thông báo khi nào hết dịch corona thì lúc đó cổ phiếu họ hàng của a quyết mới hết hot . tin chính thức từ xxunk,xxbos 5 mã trắng cửa bán rồi thì flc tí nữa thôi là lại tím lịm,xxbos hihi theo a quyết đến hết mùa dịch chắc cũng phải x2 or x3 tài khoản các bác nhỉ . tất tay hết theo a quyết chiến thôi .,xxbos mai tây ( etfs ) nó xả hàng ros , mà a qc còn chơi đẹp kéo lên cho nó xả thì các bác biết ntn rồi đấy,xxbos art , hai , klf lái con chốt , a e vô đỡ thấy thương . dự chiều 2 pm còn thảm sát nữa
y: CategoryList
1,1,1,0,0
Path: /home/tuna/.fastai/data/viwiki;

Valid: LabelList (43 items)
x: TextList
xxbos đồng ý xxunk dám xài mg,xxbos các nhóm ngành cần quan tâm theo thứ xxunk trụ vnm , vic , bvh , vcb , dhg nỗ lực phục hồi 8 phiên xxunk + hàng tiêu dùng thiết yếu msn tích lũy tạo nền được 9 tuần .bán lẻ + xxunk mwg , pnj , hcm nỗ lực phục hồi 2 phiên xxunk xxunk nỗ lực phục hồi 3 phiên xxunk , vật liệu xd ctd , xx

In [13]:
@np_func
def f1(inp,targ): return f1_score(targ, np.argmax(inp, axis=-1))
#
def auroc_score(input, target):
    input, target = input.cpu().numpy()[:,1], target.cpu().numpy()
    return roc_auc_score(target, input)

class AUROC(Callback):
    _order = -20 #Needs to run before the recorder

    def __init__(self, learn, **kwargs): self.learn = learn
    def on_train_begin(self, **kwargs): self.learn.recorder.add_metric_names(['AUROC'])
    def on_epoch_begin(self, **kwargs): self.output, self.target = [], []
    
    def on_batch_end(self, last_target, last_output, train, **kwargs):
        if not train:
            self.output.append(last_output)
            self.target.append(last_target)
                
#     def on_epoch_end(self, last_target, last_output, **kwargs):
#         if len(self.output) > 0:
#             output = torch.cat(self.output)
#             target = torch.cat(self.target)
#             preds = F.softmax(output, dim=1)
#             metric = auroc_score(preds, target)
#             self.learn.recorder.add_metrics([metric])
            
    def on_epoch_end(self, last_metrics, **kwargs):
        if len(self.output) > 0:
            output = torch.cat(self.output)
            target = torch.cat(self.target)
            preds = F.softmax(output, dim=1)
            metric = auroc_score(preds, target)
            return add_metrics(last_metrics, [metric])

In [14]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, metrics=[accuracy,f1], callback_fns=AUROC).to_fp16()
learn_c.load_encoder('stock_sentiment_finetuned_enc')
learn_c.freeze()

In [15]:
lr=2e-2
lr *= bs/48

In [16]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,AUROC,time
0,0.697207,0.602508,0.627907,0.771429,0.851852,00:00
1,0.576193,0.598596,0.627907,0.771429,0.905093,00:00


In [17]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,AUROC,time
0,0.410394,0.696452,0.627907,0.771429,0.901620,00:00
1,0.353041,0.658944,0.651163,0.782609,0.842593,00:00


In [18]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,AUROC,time
0,0.302256,0.539344,0.720930,0.812500,0.849537,00:00
1,0.286648,0.585698,0.697674,0.800000,0.872685,00:00


In [19]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,AUROC,time
0,0.228009,0.586240,0.720930,0.793103,0.821759,00:00
1,0.197966,0.663282,0.744186,0.819672,0.847222,00:00


In [20]:
learn_c.unfreeze()
learn_c.fit_one_cycle(1, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,f1,AUROC,time
0,0.117763,0.614612,0.790698,0.847458,0.854167,00:01


In [21]:
learn_c.export('stock_sentiment_model.pkl')